# Sample chatbot response

In [2]:
from typing import List, Dict, Tuple, Optional, Any
import json

In [3]:
from ollama import chat
from ollama import ChatResponse
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)

MODEL_NAME: str = "tinyllama"


class LLM:
    def __init__(self, model_name: str) -> None:
        self.model_name = model_name

    def chat(self, messages: List[Dict[str, str]]) -> str:
        response: ChatResponse = chat(
            model=self.model_name,
            messages=messages,
        )
        return response.message.content

    def chat_with_user(
        self, user_message: str, context: Optional[List[str]] = None
    ) -> str:
        context_text: str = ""
        if context is not None:
            context_text = "\n".join(context)

        response: ChatResponse = chat(
            model=self.model_name,
            messages=[
                {
                    "role": "user",
                    "content": f"{user_message}, You have some good content to frame your answer. Use the following data:  {context_text}",
                },
            ],
        )
        return response.message.content


# Sample usage.

llm = LLM(model_name=MODEL_NAME)
print(
    llm.chat_with_user(
        "My dog died few days ago. I am feeling void. What should I do to fill my void.!"
    )
)

2025-03-24 05:16:28,019 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Response: It's understandable how you must be feeling, but there are a few ways you can help yourself fill the void left by your beloved furry friend:

1. Engage in hobbies or activities that bring joy to your heart and mind. This could be anything from reading books or listening to music to painting or cooking. You may also consider volunteering at an animal shelter or helping out with household chores.

2. Focus on memories you had together, such as having long walks, playing games, and talking about old times. Consider taking photos, recording videos, or creating a scrapbook to preserve those special moments.

3. Reach out to friends and family for support and companionship. They can help you cope with your loss and provide valuable emotional support. You could also consider reaching out to professional counselors or therapists who specialize in grief support.

4. Practice mindfulness techniques like meditation, yoga, or deep breathing exercises. This can help reduce stress and anxi

# DEMO rank ranking algorithms for documents.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


class DocumentRanker:
    def __init__(self, documents: List[Dict[str, str]]) -> None:
        """
        Initialize the DocumentRanker with a list of documents.
        Each document should be a dictionary with 'id' and 'content' keys.
        """
        self.documents: List[Dict[str, str]] = documents
        self.vectorizer: CountVectorizer = CountVectorizer()
        self.doc_term_matrix = self.vectorizer.fit_transform(
            [doc["problem"] for doc in documents]
        )

    def rank_documents(
        self, query: str, limit: int = 3
    ) -> List[Tuple[Dict[str, str], float]]:
        """
        Rank documents based on their relevance to the query using cosine similarity.
        :param query: The query string to compare against the documents.
        :param limit: The number of top documents to return.
        :return: A list of ranked documents with their scores.
        """
        query_vector = self.vectorizer.transform([query])
        scores = cosine_similarity(query_vector, self.doc_term_matrix).flatten()
        ranked_documents = sorted(
            zip(self.documents, scores), key=lambda x: x[1], reverse=True
        )
        return ranked_documents[:limit]

    def display_ranked_documents(
        self, ranked_documents: List[Tuple[Dict[str, str], float]]
    ) -> None:
        """
        Display the ranked documents in a readable format.
        :param ranked_documents: A list of tuples containing documents and their scores.
        """
        print("Ranked Documents:")
        for rank, (doc, score) in enumerate(ranked_documents, start=1):
            print(f"Rank {rank}: Document ID {doc['id']} with score {score:.4f}")
            print(f"Content: {doc['solution']}")
            print()

In [5]:
# Sample demonstration of the DocumentRanker class

documents: List[Dict[str, Any]] = []

with open("files/docs.json", "r") as json_file:
    data = json.load(json_file)
    for doc in data:
        documents.append(doc)

query = "I am feeling lost and overwhelmed. How can I find my way back?"

ranker = DocumentRanker(documents)
ranked_docs = ranker.rank_documents(query)
ranker.display_ranked_documents(ranked_docs)

Ranked Documents:
Rank 1: Document ID 6 with score 0.4743
Content: Solution: Treat yourself as you would treat a friend. Practice self-compassion by using kind words and forgiving yourself for mistakes.

Rank 2: Document ID 9 with score 0.3922
Content: Solution: Remind yourself of past obstacles you have overcome. Break challenges into manageable steps and tackle them one at a time.

Rank 3: Document ID 10 with score 0.3922
Content: Solution: Identify what is within your control and direct your energy towards it. Practice letting go of things you cannot change through meditation or affirmations.



# Add vector semantic search 

In [6]:
import logging
from collections import OrderedDict
import uuid

from qdrant_client import QdrantClient, models
from qdrant_client.models import PointStruct
from qdrant_client.http.models import VectorParams, Distance
import ollama


class SemanticEmbeddingService:
    def __init__(self, cache_size: int = 1000) -> None:
        self.embeddings_cache: OrderedDict[str, list[float]] = OrderedDict()
        self.cache_size = cache_size

    def get_embeddings(self, text: str) -> list[float]:
        if text in self.embeddings_cache:
            self.embeddings_cache.move_to_end(text)
            return self.embeddings_cache[text]
        else:
            response = ollama.embed(model="mxbai-embed-large", input=text)
            self.embeddings_cache[text] = response.embeddings[0]
            logging.info(f"Embedding for {text} is {str(response.embeddings[0])}")
            if len(self.embeddings_cache) > self.cache_size:
                self.embeddings_cache.popitem(last=False)

            return self.embeddings_cache[text]

In [7]:
# Initialize the SemanticEmbeddingService with a cache size of 1000
embedding_service = SemanticEmbeddingService(cache_size=1000)

# Example text to generate embeddings for
text = "How are you feeling today?"

# Get embeddings for the text
embeddings = embedding_service.get_embeddings(text)

# Print the embeddings
print(f"Embeddings for '{text}': {embeddings}")

2025-03-24 05:16:30,248 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-24 05:16:30,249 [INFO] Embedding for How are you feeling today? is [0.043686878, 0.010882682, -0.06962838, 0.00946688, -0.050298635, -0.06430906, 0.055805095, 0.03489373, 0.029922625, 0.007141791, 0.0086399615, -0.0011330395, -0.023501214, -0.0070877923, -0.052488104, -0.05390703, -0.013800829, -0.01061243, -0.0113593, 0.021342738, -0.055232637, 0.043069627, -0.03797889, 0.043831967, -0.048233528, 0.030241698, -0.010079717, -0.023305472, 0.014025283, 0.044474613, -0.059587717, 0.033727024, 0.014366324, -0.03741475, 0.02853252, -0.03804429, 0.021235922, -0.032757875, 0.018696612, -0.04434006, -0.0053693634, 0.0078080837, 0.014779382, -0.027833888, -0.052784692, -0.026691355, -0.035743322, -0.0481956, 0.019810278, -0.049065787, 0.016198859, 0.00077878265, -0.014836713, -0.016452556, 0.035978243, 0.003948995, -0.030972108, 0.056124236, -0.04835454, 0.019671641, 0.03162803, 0.006236

Embeddings for 'How are you feeling today?': [0.043686878, 0.010882682, -0.06962838, 0.00946688, -0.050298635, -0.06430906, 0.055805095, 0.03489373, 0.029922625, 0.007141791, 0.0086399615, -0.0011330395, -0.023501214, -0.0070877923, -0.052488104, -0.05390703, -0.013800829, -0.01061243, -0.0113593, 0.021342738, -0.055232637, 0.043069627, -0.03797889, 0.043831967, -0.048233528, 0.030241698, -0.010079717, -0.023305472, 0.014025283, 0.044474613, -0.059587717, 0.033727024, 0.014366324, -0.03741475, 0.02853252, -0.03804429, 0.021235922, -0.032757875, 0.018696612, -0.04434006, -0.0053693634, 0.0078080837, 0.014779382, -0.027833888, -0.052784692, -0.026691355, -0.035743322, -0.0481956, 0.019810278, -0.049065787, 0.016198859, 0.00077878265, -0.014836713, -0.016452556, 0.035978243, 0.003948995, -0.030972108, 0.056124236, -0.04835454, 0.019671641, 0.03162803, 0.0062368033, -0.012064472, -0.08383989, -0.018723916, 0.0024015629, -0.02351107, -0.011221358, -0.00914808, -0.018726422, -0.0476745, 0.00

In [8]:
COLLECTION_NAME = "emotional_chatbot"


class SemanticQdrantService:
    def __init__(self, url: str) -> None:
        self.client = QdrantClient(url=url)

    def collection_exists(self, collection_name) -> bool:
        try:
            response = self.client.get_collection(collection_name)
            return response is not None
        except Exception as e:
            return False

    def create_collection(self, collection_name: str) -> None:
        print(
            f"If the collection {collection_name} does not exist, create it. {self.collection_exists(collection_name)}"
        )
        if self.collection_exists(collection_name) is False:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
            )

    def delete_collection(self, collection_name: str) -> None:
        if self.collection_exists(collection_name):
            self.client.delete_collection(collection_name)

    def upsert_points(self, collection_name: str, points: list[PointStruct]) -> None:
        self.client.upsert(collection_name=collection_name, points=points)

    def search(self, query_embedding: list[float], limit: int = 3) -> list[PointStruct]:
        return self.client.search(
            collection_name=COLLECTION_NAME,
            query_vector=query_embedding,
            limit=limit,
        )

In [9]:
# Initialize the service
url = "http://localhost:6333"  # Replace with your Qdrant URL
service = SemanticQdrantService(url=url)

# Create a collection if it doesn't exist
collection_name = "emotional_chatbot"
service.delete_collection(collection_name)
service.create_collection(collection_name)

2025-03-24 05:16:30,303 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-24 05:16:30,313 [INFO] HTTP Request: GET http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 200 OK"
2025-03-24 05:16:30,346 [INFO] HTTP Request: DELETE http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 200 OK"
2025-03-24 05:16:30,351 [INFO] HTTP Request: GET http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 404 Not Found"
2025-03-24 05:16:30,352 [INFO] HTTP Request: GET http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 404 Not Found"


If the collection emotional_chatbot does not exist, create it. False


2025-03-24 05:16:30,528 [INFO] HTTP Request: PUT http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 200 OK"


In [10]:
# Sample usage of the SemanticQdrantService class


from qdrant_client.models import PointStruct


def use_semantic_qdrant_service():
    # Initialize the embedding service
    embedding_service = SemanticEmbeddingService(cache_size=1000)

    # Example texts to generate embeddings for
    text1 = "How are you feeling today?"
    text2 = "What is your emotional state?"

    # Get embeddings for the texts
    embedding1 = embedding_service.get_embeddings(text1)
    embedding2 = embedding_service.get_embeddings(text2)

    # Initialize the Qdrant service
    url = "http://localhost:6333"  # Replace with your Qdrant URL
    service = SemanticQdrantService(url=url)

    # Create a collection if it doesn't exist
    collection_name = "emotional_chatbot"
    service.create_collection(collection_name)

    # Upsert points into the collection
    points = [
        PointStruct(
            id=1,
            vector=embedding1,  # Use generated embedding
            payload={
                "solution": "Solution: Treat yourself as you would treat a friend. Practice self-compassion by using kind words and forgiving yourself for mistakes.",
                "problem": "Problem: Negative self-talk and self-criticism can make you feel unworthy of care and support.",
            },
        ),
    ]
    service.upsert_points(collection_name=collection_name, points=points)

    # Perform a search
    query_text = "How are you feeling today?"
    query_embedding = embedding_service.get_embeddings(query_text)
    results = service.search(query_embedding=query_embedding, limit=5)

    # Print the search results
    for result in results:
        print(f"ID: {result.id}, Score: {result.score}, Payload: {result.payload}")


# Call the function
use_semantic_qdrant_service()

2025-03-24 05:16:30,805 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-24 05:16:30,806 [INFO] Embedding for How are you feeling today? is [0.043686878, 0.010882682, -0.06962838, 0.00946688, -0.050298635, -0.06430906, 0.055805095, 0.03489373, 0.029922625, 0.007141791, 0.0086399615, -0.0011330395, -0.023501214, -0.0070877923, -0.052488104, -0.05390703, -0.013800829, -0.01061243, -0.0113593, 0.021342738, -0.055232637, 0.043069627, -0.03797889, 0.043831967, -0.048233528, 0.030241698, -0.010079717, -0.023305472, 0.014025283, 0.044474613, -0.059587717, 0.033727024, 0.014366324, -0.03741475, 0.02853252, -0.03804429, 0.021235922, -0.032757875, 0.018696612, -0.04434006, -0.0053693634, 0.0078080837, 0.014779382, -0.027833888, -0.052784692, -0.026691355, -0.035743322, -0.0481956, 0.019810278, -0.049065787, 0.016198859, 0.00077878265, -0.014836713, -0.016452556, 0.035978243, 0.003948995, -0.030972108, 0.056124236, -0.04835454, 0.019671641, 0.03162803, 0.006236

If the collection emotional_chatbot does not exist, create it. True
ID: 1, Score: 0.9999998, Payload: {'solution': 'Solution: Treat yourself as you would treat a friend. Practice self-compassion by using kind words and forgiving yourself for mistakes.', 'problem': 'Problem: Negative self-talk and self-criticism can make you feel unworthy of care and support.'}


In [11]:
class SemanticSearchRepo:
    def __init__(
        self,
        embedding_service: SemanticEmbeddingService,
        qdrant_service: SemanticQdrantService,
    ):
        self.embedding_service = embedding_service
        self.qdrant_service = qdrant_service

    def prepare_points(
        self, texts: list[str], metadata: list[dict]
    ) -> list[PointStruct]:
        return [
            PointStruct(
                id=str(uuid.uuid4()),
                vector=self.embedding_service.get_embeddings(text),
                payload={"text": text, **meta},
            )
            for _, (text, meta) in enumerate(zip(texts, metadata))
        ]

    async def create_collection(self, collection_name: str):
        collection_exists = self.qdrant_service.client.collection_exists(
            collection_name=collection_name
        )
        if collection_exists is False:
            self.qdrant_service.client.create_collection(
                collection_name,
                vectors_config=models.VectorParams(
                    size=1024, distance=models.Distance.COSINE
                ),
            )

    def initialize_qdrant(
        self, texts: list[str], metadata: list[dict[str, str]]
    ) -> bool:
        points = self.prepare_points(texts, metadata)
        self.qdrant_service.upsert_points(COLLECTION_NAME, points)
        return True

    def query_text(self, query_text: str) -> list[dict]:
        query_embedding = self.embedding_service.get_embeddings(query_text)
        response = self.qdrant_service.search(query_embedding)
        logging.info(f"Query: {query_text}")
        result = []

        logging.info(f"Response: {response}")
        for data in response:
            if data.score > 0.5:
                result.append(
                    {
                        "score": data.score,
                        "solution": data.payload["solution"],
                        "metadata": data.payload,
                    }
                )
        return result

In [12]:
with open("files/docs.json", "r") as json_file:
    data = json.load(json_file)
    for doc in data:
        documents.append(doc)

    # Initialize the required services
embedding_service = SemanticEmbeddingService()
qdrant_service = SemanticQdrantService(url=url)

# Create an instance of SemanticSearchRepo
semantic_repo = SemanticSearchRepo(embedding_service, qdrant_service)


def sample_usage():
    # Prepare the data
    metadata: List[Dict[str, str]] = [
        {"solution": item["solution"], "problem": item["problem"]} for item in documents
    ]
    texts: List[str] = [item["problem"] for item in metadata]
    # # Initialize Qdrant with the sample data
    semantic_repo.initialize_qdrant(texts, metadata)


sample_usage()

2025-03-24 05:16:31,245 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-24 05:16:32,056 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-24 05:16:32,058 [INFO] Embedding for Problem: Sometimes, you might feel weak or incapable of handling life's challenges. Doubt and fear can make you question your own resilience. is [-0.012015247, -0.039253023, 0.0026042445, 0.055119123, 0.0011163378, -0.045438852, 0.0198595, 0.02442931, 0.019776935, 0.027262023, 0.022706665, -0.032893054, 0.019327503, -0.04369738, 0.012364698, 0.013371439, -0.028224522, -0.0016991321, -0.033389483, 0.019733032, 0.011345365, 0.031440463, -0.01611639, -0.006411766, -0.03778136, -0.011906415, 0.0059362054, -0.013025983, 0.014672039, 0.04873149, -0.023392584, -0.011265147, -0.037335314, -0.02210455, -0.017938286, -0.017067865, 0.023042247, -0.047092475, -0.046367753, -0.018426323, -0.014793182, 0.008115606, 0.03186983, -0.018842736, -0.046648446, -0.021361578, 

In [13]:
def query():
    # Query the data
    query_text = "I am too much worried about the past present and the future."

    results = semantic_repo.query_text(query_text)

    # Print the results
    print("Query Results:")
    for result in results:
        print(result)


query()

2025-03-24 05:16:38,118 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-24 05:16:38,119 [INFO] Embedding for I am too much worried about the past present and the future. is [0.0050358637, -0.023398962, 0.00863306, 0.0335489, 0.012818076, -0.060886, 0.008563702, 0.004619987, 0.025248714, -0.012921162, 0.009773511, -0.0851847, -0.00060851517, -0.02041468, -0.015575697, 0.01812516, -0.031834304, 0.0066795377, -0.008827113, 0.026197275, 0.045282196, 0.019876538, -0.028904796, -0.049455322, -0.0067687747, 0.050773047, 0.015421382, 0.006528653, 0.018986197, 0.06509573, -0.017859515, 0.019520938, -0.017865775, 0.0022602014, -0.022991836, 0.017727999, 0.023096563, 0.034047782, 0.039686874, -0.036785655, -0.026216663, 0.0010515952, 0.07028585, -0.016376073, -0.03840195, -0.0012187443, -0.02899664, -0.05027885, 0.0327074, -0.0486368, -0.0013882788, 0.017105676, 0.002154736, 0.018429218, 0.06629947, 0.010923629, 0.025710171, 0.020914478, -0.00057111064, 0.0028

Query Results:
{'score': 0.8048525, 'solution': 'Solution: Practice mindfulness by focusing on your breathing. Engage in activities that ground you, like meditation or journaling.', 'metadata': {'text': 'Problem: Worrying about the past or future can create anxiety and prevent you from enjoying the present.', 'solution': 'Solution: Practice mindfulness by focusing on your breathing. Engage in activities that ground you, like meditation or journaling.', 'problem': 'Problem: Worrying about the past or future can create anxiety and prevent you from enjoying the present.'}}
{'score': 0.8048525, 'solution': 'Solution: Practice mindfulness by focusing on your breathing. Engage in activities that ground you, like meditation or journaling.', 'metadata': {'text': 'Problem: Worrying about the past or future can create anxiety and prevent you from enjoying the present.', 'solution': 'Solution: Practice mindfulness by focusing on your breathing. Engage in activities that ground you, like meditatio

In [14]:
# Adding vector store context to the chatbot

llm = LLM(model_name=MODEL_NAME)

query = "My dog died last week. I am feeling void. What should I do?"

context: List[str] = []

# Take out context from the demo rank
ranked_documents = ranker.rank_documents(query)
for doc, _ in ranked_documents:
    logging.info(f"Document: {doc}")
    context.append(doc["solution"])

# Take out the context from the semantic search
results = semantic_repo.query_text(query)
for idx, result in enumerate(results):
    logging.info(f"{idx} Result: {result}")
    context.append(result["solution"])

2025-03-24 05:16:38,147 [INFO] Document: {'id': 1, 'problem': "Problem: Sometimes, you might feel weak or incapable of handling life's challenges. Doubt and fear can make you question your own resilience.", 'solution': 'Solution: Remind yourself of past struggles you have overcome. Write down your achievements, no matter how small, and build confidence from them.'}
2025-03-24 05:16:38,147 [INFO] Document: {'id': 2, 'problem': 'Problem: Life can become too much to handle, and you might feel buried under stress, expectations, or emotions.', 'solution': 'Solution: Take a step back and acknowledge your feelings without judgment. Break down tasks into smaller steps and seek support from trusted people.'}
2025-03-24 05:16:38,147 [INFO] Document: {'id': 3, 'problem': 'Problem: Worrying about the past or future can create anxiety and prevent you from enjoying the present.', 'solution': 'Solution: Practice mindfulness by focusing on your breathing. Engage in activities that ground you, like med

In [ ]:
results: List[Dict[str, str]] = []

response_without_context = llm.chat_with_user(query)
logging.info(
    f"Llama response without context:\n===============================================>\n {response_without_context}"
)

results.append({"response without context": response_without_context})

response_with_context = llm.chat_with_user(query, context=context)
logging.info(
    f"Llama response with context:\n===================================================>\n {response_with_context}"
)

results.append({"response with context": response_with_context})

# Dump the results in json

with open("files/results.json", "w") as json_file:
    json.dump(results, json_file, indent=2)

2025-03-24 05:18:46,036 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-24 05:18:46,036 [INFO] Llama response without context:
===============================================>
 Dear User,

As a compassionate AI assistant, I understand how much you're missing your beloved dog. I can empathize with your void. However, as you are aware of your loss, here are some practical suggestions to help you cope with this difficult time:

1. Take some time to grieve - It's okay to feel sad or emotional. Grieving for a loved one is a natural part of the healing process, and it's perfectly normal to experience feelings of sadness, anger, and other emotions. Take some time off work or school to spend some quality time with your thoughts, taking deep breaths, or just sitting in silence.

2. Spend time with pets - Your dog was always there for you, and they'll never leave. Consider getting a new pet as a substitute for the lost one. Pets offer comfort, companionship, a

# Add memory to the application